<a href="https://colab.research.google.com/github/saiku122/AIJobcolle/blob/master/MachineLearning/python/exercise/02_work/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/saiku122/AIJobcolle.git

fatal: destination path 'AIJobcolle' already exists and is not an empty directory.


In [3]:
cd /content/AIJobcolle/MachineLearning/python

/content/AIJobcolle/MachineLearning/python


In [5]:
cd ./exercise/02_work/

/content/AIJobcolle/MachineLearning/python/exercise/02_work


In [9]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.externals import joblib

def main():
  # data load
  df = pd.read_csv('./data/'+ file_model + '.csv', header=0)
  ID = df.iloc[:,0] 
  y = df.iloc[:,-1]
  X = df.iloc[:,1:-1]

  # preprocessing-1: one-hot encoding
  X_ohe = pd.get_dummies(X, dummy_na=True, columns=ohe_cols)
  X_ohe = X_ohe.dropna(axis=1, how='all')
  X_ohe_columns = X_ohe.columns.values

  # preprocessing-2: null imputation
  imp = SimpleImputer()
  imp.fit(X_ohe)
  X_ohe = pd.DataFrame(imp.transform(X_ohe), columns=X_ohe_columns)
  print(X_ohe.shape)

  # preprocessing-3: feature selection
  selector = RFECV(estimator=RandomForestClassifier(n_estimators=100,random_state=0), step=0.05)
  selector.fit(X_ohe, y)
  X_ohe_selected = selector.transform(X_ohe)
  X_ohe_selected = pd.DataFrame(X_ohe_selected, columns=X_ohe_columns[selector.support_])
  print(X_ohe_selected.shape)
  X_ohe_selected.head()

  # preprocessing-4: preprocessing of a score data along with a model dataset
  if len(file_score)>0:
      # load score data
      dfs = pd.read_csv('./data/'+ file_score + '.csv', header=0)
      IDs = dfs.iloc[:,[0]] 
      Xs = dfs.iloc[:,1:-1]
      Xs_ohe = pd.get_dummies(Xs, dummy_na=True, columns=ohe_cols)
      cols_m = pd.DataFrame(None, columns=X_ohe_columns, dtype=float)

      # consistent with columns set
      Xs_exp = pd.concat([cols_m, Xs_ohe])
      Xs_exp.loc[:,list(set(X_ohe_columns)-set(Xs_ohe.columns.values))] = \
          Xs_exp.loc[:,list(set(X_ohe_columns)-set(Xs_ohe.columns.values))].fillna(0, axis=1)
      Xs_exp = Xs_exp.drop(list(set(Xs_ohe.columns.values)-set(X_ohe_columns)), axis=1)

      # re-order the score data columns
      Xs_exp = Xs_exp.reindex_axis(X_ohe_columns, axis=1)
      Xs_exp = pd.DataFrame(imp.transform(Xs_exp), columns=X_ohe_columns)
      Xs_exp_selected = Xs_exp.loc[:, X_ohe_columns[selector.support_]]

  # modeling
  clf.fit(X_ohe_selected, y.as_matrix().ravel())
  joblib.dump(clf, './model/'+ model_name + '.pkl')
  results = cross_val_score(clf, X_ohe_selected, y, scoring='roc_auc', cv=5)
  print('cv score:', np.average(results), '+-', np.std(results))

  # scoring
  if len(file_score)>0:
      score = pd.DataFrame(clf.predict_proba(Xs_exp_selected)[:,1], columns=['pred_score'])
      IDs.join(score).to_csv('./data/'+  model_name + '_' + file_score + '_with_pred.csv', index=False)

  # model profile
  imp = pd.DataFrame([clf.named_steps['est'].feature_importances_], columns=X_ohe_columns[selector.support_])
  imp.T.to_csv('./data/'+  model_name + '_feature_importances.csv', index=True)

if __name__ == '__main__':

  # SET PARAMETERS
  file_model = 'dm_for_model'
  file_score = 'dm_for_fwd'
  ohe_cols = ['mode_category']

  # CLASSIFIER
  model_name = 'GBC_001'
  clf = Pipeline([('scl',StandardScaler()),('est',GradientBoostingClassifier(random_state=1))])

  # お手数ですがコメントを外して別途実行下さい
  # model_name = 'RF_001'
  # clf = Pipeline([('scl',StandardScaler()), ('est',RandomForestClassifier(random_state=1))])

  # MAIN PROC
  main()


(717, 334)
(717, 318)


AttributeError: ignored